This notebook is based on my previous notebook [Baseline_For_Beginners](https://www.kaggle.com/code/adamlogman/baseline-for-beginners),and [Dmitry Uarov](https://www.kaggle.com/dmitryuarov) 's [NeurIPS | Baseline + External data](https://www.kaggle.com/code/dmitryuarov/neurips-baseline-external-data) notebook with modifications to the model

# Import Dependencies 

In [1]:
!pip install /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl
!pip install mordred --no-index --find-links=file:///kaggle/input/mordred-1-2-0-py3-none-any/

Processing /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl
Looking in links: file:///kaggle/input/mordred-1-2-0-py3-none-any/
Processing /kaggle/input/mordred-1-2-0-py3-none-any/mordred-1.2.0-py3-none-any.whl
Processing /kaggle/input/mordred-1-2-0-py3-none-any/networkx-2.8.8-py3-none-any.whl (from mordred)
  Attempting uninstall: networkx
    Found existing installation: networkx 3.4.2
    Uninstalling networkx-3.4.2:
      Successfully uninstalled networkx-3.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.25.2 requires networkx>=3.0, but you have networkx 2.8.8 which is incompatible.
nx-cugraph-cu12 25.2.0 requires networkx>=3.2, but you have networkx 2.8.8 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but yo

In [2]:
 # Importing Required Libraries\nLet's begin by importing the essential Python libraries needed for data processing, visualization, and modeling.

import pandas as pd
import numpy as np


from sklearn.ensemble import HistGradientBoostingRegressor,ExtraTreesRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from mordred import Calculator, descriptors
from sklearn.feature_selection import RFE
import networkx as nx
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdmolops
from rdkit import Chem

from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import (
    RandomForestRegressor,
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    HistGradientBoostingRegressor
)
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [3]:
class CFG:
    TARGETS = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
    SEED = 42
    FOLDS = 10

In [4]:
useless_cols = [   
    
    'MaxPartialCharge', 
    # Nan data
    'BCUT2D_MWHI',
    'BCUT2D_MWLOW',
    'BCUT2D_CHGHI',
    'BCUT2D_CHGLO',
    'BCUT2D_LOGPHI',
    'BCUT2D_LOGPLOW',
    'BCUT2D_MRHI',
    'BCUT2D_MRLOW',

    # Constant data
    'NumRadicalElectrons',
    'SMR_VSA8',
    'SlogP_VSA9',
    'fr_barbitur',
    'fr_benzodiazepine',
    'fr_dihydropyridine',
    'fr_epoxide',
    'fr_isothiocyan',
    'fr_lactam',
    'fr_nitroso',
    'fr_prisulfonamd',
    'fr_thiocyan',

    # High correlated data >0.95
    'MaxEStateIndex',
    'HeavyAtomMolWt',
    'ExactMolWt',
    'NumValenceElectrons',
    'Chi0',
    'Chi0n',
    'Chi0v',
    'Chi1',
    'Chi1n',
    'Chi1v',
    'Chi2n',
    'Kappa1',
    'LabuteASA',
    'HeavyAtomCount',
    'MolMR',
    'Chi3n',
    'BertzCT',
    'Chi2v',
    'Chi4n',
    'HallKierAlpha',
    'Chi3v',
    'Chi4v',
    'MinAbsPartialCharge',
    'MinPartialCharge',
    'MaxAbsPartialCharge',
    'FpDensityMorgan2',
    'FpDensityMorgan3',
    'Phi',
    'Kappa3',
    'fr_nitrile',
    'SlogP_VSA6',
    'NumAromaticCarbocycles',
    'NumAromaticRings',
    'fr_benzene',
    'VSA_EState6',
    'NOCount',
    'fr_C_O',
    'fr_C_O_noCOO',
    'NumHDonors',
    'fr_amide',
    'fr_Nhpyrrole',
    'fr_phenol',
    'fr_phenol_noOrthoHbond',
    'fr_COO2',
    'fr_halogen',
    'fr_diazo',
    'fr_nitro_arom',
    'fr_phos_ester'
]

SEED = 42

# Read Files

### Main Files

In [5]:
 #We will load both the training and test datasets using pandas, and store test IDs 
train=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train.csv')
test=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/test.csv')
ss=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/sample_submission.csv')
ID=test['id']

In [6]:
mols_test = [Chem.MolFromSmiles(s) for s in test.SMILES]

# Initialize the Mordred Calculator
calc = Calculator(descriptors, ignore_3D=True) # ignore_3D=True for 2D descriptors

desc_test = calc.pandas(mols_test)
desc_test.shape

100%|██████████| 3/3 [00:00<00:00,  4.15it/s]


(3, 1613)

In [7]:
tc_smiles =pd.read_csv('/kaggle/input/tc-smiles/Tc_SMILES.csv',      low_memory=False)
tg_smiles =pd.read_csv('/kaggle/input/smiles-extra-data/JCIM_sup_bigsmiles.csv',      low_memory=False)
ktg_smiles =pd.read_excel('/kaggle/input/smiles-extra-data/data_tg3.xlsx')
de_smiles =pd.read_excel('/kaggle/input/smiles-extra-data/data_dnst1.xlsx')
ds1_smiles=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset1.csv',      low_memory=False)
ds2_smiles=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset2.csv',      low_memory=False)
ds3_smiles=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset3.csv',      low_memory=False)
ds4_smiles=pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv',      low_memory=False)

tg_ds      = pd.read_csv('/kaggle/input/modred-dataset/desc_tg.csv',      low_memory=False)
tc_ds      = pd.read_csv('/kaggle/input/modred-dataset/desc_tc.csv',      low_memory=False)
rg_ds      = pd.read_csv('/kaggle/input/modred-dataset/desc_rg.csv',      low_memory=False)
ffv_ds     = pd.read_csv('/kaggle/input/modred-dataset/desc_ffv.csv',     low_memory=False)
den_ds     = pd.read_csv('/kaggle/input/modred-dataset/desc_de.csv',      low_memory=False)

### Extra Files

# Preprocessing 

In [8]:
for i in (tg_ds,tc_ds,rg_ds,ffv_ds,den_ds):
     i.drop(columns=[col for col in i.columns if i[col].nunique() == 1],axis=1,inplace=True)

tg_ds = tg_ds.select_dtypes(exclude=['object', 'category'])
rg_ds = rg_ds.select_dtypes(exclude=['object', 'category'])
ffv_ds = ffv_ds.select_dtypes(exclude=['object', 'category'])
tc_ds = tc_ds.select_dtypes(exclude=['object', 'category'])
den_ds = den_ds.select_dtypes(exclude=['object', 'category'])

In [9]:
def make_smile_canonical(smile): # To avoid duplicates, for example: canonical '*C=C(*)C' == '*C(=C*)C'
    try:
        mol = Chem.MolFromSmiles(smile)
        canon_smile = Chem.MolToSmiles(mol, canonical=True)
        return canon_smile
    except:
        return np.nan

train['SMILES'] = train['SMILES'].apply(lambda s: make_smile_canonical(s))
test['SMILES'] = test['SMILES'].apply(lambda s: make_smile_canonical(s))

In [10]:
ktg_smiles.rename(columns={'Tg [K]': 'Tg'}, inplace=True)
tg_smiles.rename(columns={'Tg (C)': 'Tg'}, inplace=True)
tc_smiles.rename(columns={'TC_mean': 'Tc'}, inplace=True)
de_smiles.rename(columns={'density(g/cm3)': 'Density'}, inplace=True)
ds1_smiles.rename(columns={'TC_mean': 'Tc'}, inplace=True)

In [11]:
de_smiles['SMILES'] = de_smiles['SMILES'].apply(lambda s: make_smile_canonical(s))
de_smiles = de_smiles[(de_smiles['SMILES'].notnull())&(de_smiles['Density'].notnull())&(de_smiles['Density'] != 'nylon')]
de_smiles['Density'] = de_smiles['Density'].astype('float64')
de_smiles['Density'] -= 0.118

ktg_smiles['Tg'] = ktg_smiles['Tg'] - 273.15

[14:00:46] SMILES Parse Error: syntax error while parsing: *O[Si](*)([R])[R]
[14:00:46] SMILES Parse Error: check for mistakes around position 12:
[14:00:46] *O[Si](*)([R])[R]
[14:00:46] ~~~~~~~~~~~^
[14:00:46] SMILES Parse Error: Failed parsing SMILES '*O[Si](*)([R])[R]' for input: '*O[Si](*)([R])[R]'
[14:00:46] SMILES Parse Error: syntax error while parsing: *NC(=O)c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=O)c3c4
[14:00:46] SMILES Parse Error: check for mistakes around position 28:
[14:00:46] c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=
[14:00:46] ~~~~~~~~~~~~~~~~~~~~^
[14:00:46] SMILES Parse Error: Failed parsing SMILES '*NC(=O)c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=O)c3c4' for input: '*NC(=O)c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=O)c3c4'
[14:00:46] SMILES Parse Error: syntax error while parsing: O=C=N[R1]N=C=O.O[R2]O.O[R3]O
[14:00:46] SMILES Parse Error: check for mistakes around position 7:
[14:00:46] O=C=N[R1]N=C=O.O[R2]O.O[R3]O
[14:00:46] ~~~~~~^
[14:00:46] SMILES Parse Error: F

In [12]:
def preprocessing(df):
    desc_names = [desc[0] for desc in Descriptors.descList if desc[0] not in useless_cols]
    descriptors = [compute_all_descriptors(smi) for smi in df['SMILES'].to_list()]

    graph_feats = {'graph_diameter': [], 'avg_shortest_path': [], 'num_cycles': []}
    for smile in df['SMILES']:
         compute_graph_features(smile, graph_feats)
        
    result = pd.concat(
        [
            pd.DataFrame(descriptors, columns=desc_names),
            pd.DataFrame(graph_feats)
        ],
        axis=1
    )

    result = result.replace([-np.inf, np.inf], np.nan)
    return result

# Feature Extraction 

In [13]:
# SMILESを正規化（canonical化）
# 重複する SMILES を集約して平均 target 値に
# df_train に存在するが欠損している target に、df_extra から値を補完

def add_extra_data(df_train, df_extra, target):
    n_samples_before = len(df_train[df_train[target].notnull()])
    
    df_extra['SMILES'] = df_extra['SMILES'].apply(lambda s: make_smile_canonical(s))
    df_extra = df_extra.groupby('SMILES', as_index=False)[target].mean()
    cross_smiles = set(df_extra['SMILES']) & set(df_train['SMILES'])
    unique_smiles_extra = set(df_extra['SMILES']) - set(df_train['SMILES'])

    # Make priority target value from competition's df
    for smile in df_train[df_train[target].notnull()]['SMILES'].tolist():
        if smile in cross_smiles:
            cross_smiles.remove(smile)

    # Imput missing values for competition's SMILES
    for smile in cross_smiles:
        df_train.loc[df_train['SMILES']==smile, target] = df_extra[df_extra['SMILES']==smile][target].values[0]
    
    df_train = pd.concat([df_train, df_extra[df_extra['SMILES'].isin(unique_smiles_extra)]], axis=0).reset_index(drop=True)

    n_samples_after = len(df_train[df_train[target].notnull()])
    print(f'\nFor target "{target}" added {n_samples_after-n_samples_before} new samples!')
    print(f'New unique SMILES: {len(unique_smiles_extra)}')
    return df_train

train = add_extra_data(train, tc_smiles, 'Tc')
train = add_extra_data(train, tg_smiles, 'Tg')
train = add_extra_data(train, ktg_smiles, 'Tg')
train = add_extra_data(train, de_smiles, 'Density')

train = add_extra_data(train, ds1_smiles, 'Tc')
#train = add_extra_data(train, ds2_smiles, 'Density')
train = add_extra_data(train, ds3_smiles, 'Tg')
train = add_extra_data(train, ds4_smiles, 'FFV')


For target "Tc" added 129 new samples!
New unique SMILES: 129

For target "Tg" added 151 new samples!
New unique SMILES: 136

For target "Tg" added 499 new samples!
New unique SMILES: 499

For target "Density" added 634 new samples!
New unique SMILES: 524

For target "Tc" added 0 new samples!
New unique SMILES: 0

For target "Tg" added 0 new samples!
New unique SMILES: 0

For target "FFV" added 862 new samples!
New unique SMILES: 819


In [14]:
def compute_all_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [None] * len(desc_names)
    return [desc[1](mol) for desc in Descriptors.descList if desc[0] not in useless_cols]

def compute_graph_features(smiles, graph_feats):
    mol = Chem.MolFromSmiles(smiles)
    adj = rdmolops.GetAdjacencyMatrix(mol)
    G = nx.from_numpy_array(adj)

    graph_feats['graph_diameter'].append(nx.diameter(G) if nx.is_connected(G) else 0)
    graph_feats['avg_shortest_path'].append(nx.average_shortest_path_length(G) if nx.is_connected(G) else 0)
    graph_feats['num_cycles'].append(len(list(nx.cycle_basis(G))))

train = pd.concat([train, preprocessing(train)], axis=1)
test = pd.concat([test, preprocessing(test)], axis=1)

# Find constant columns for each target
all_features = train.columns[7:].tolist()
features = {}
for target in CFG.TARGETS:
    const_descs = []
    for col in train.columns.drop(CFG.TARGETS):
        if train[train[target].notnull()][col].nunique() == 1:
            const_descs.append(col)
    features[target] = [f for f in all_features if f not in const_descs]

print(train.shape)
train['Ipc']=np.log10(train['Ipc'])  
for n in train.columns[7:]:
    train[n]=train[n].replace(-np.inf,np.nan)
    train[n]=train[n].replace(np.inf,np.nan)    
    train[n].fillna(train[n].mean())
  
print(train.shape)
test['Ipc']=np.log10(test['Ipc'])
for n in test.columns[7:]:
    train[n]=train[n].replace(-np.inf,np.nan)
    train[n]=train[n].replace(np.inf,np.nan)      
    test[n].fillna(train[n].mean())

(10080, 158)
(10080, 158)


In [15]:
tg_ds.shape

(9117, 594)

In [16]:
train

,id,SMILES,Tg,FFV,Tc,Density,Rg,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,FpDensityMorgan1,AvgIpc,BalabanJ,Ipc,Kappa2,PEOE_VSA1,PEOE_VSA10,PEOE_VSA11,PEOE_VSA12,PEOE_VSA13,PEOE_VSA14,PEOE_VSA2,PEOE_VSA3,PEOE_VSA4,PEOE_VSA5,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,PEOE_VSA9,SMR_VSA1,SMR_VSA10,SMR_VSA2,SMR_VSA3,SMR_VSA4,SMR_VSA5,SMR_VSA6,SMR_VSA7,SMR_VSA9,SlogP_VSA1,SlogP_VSA10,SlogP_VSA11,SlogP_VSA12,SlogP_VSA2,SlogP_VSA3,SlogP_VSA4,SlogP_VSA5,SlogP_VSA7,SlogP_VSA8,TPSA,EState_VSA1,EState_VSA10,EState_VSA11,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,EState_VSA6,EState_VSA7,EState_VSA8,EState_VSA9,VSA_EState1,VSA_EState10,VSA_EState2,VSA_EState3,VSA_EState4,VSA_EState5,VSA_EState7,VSA_EState8,VSA_EState9,FractionCSP3,NHOHCount,NumAliphaticCarbocycles,NumAliphaticHeterocycles,NumAliphaticRings,NumAmideBonds,NumAromaticHeterocycles,NumAtomStereoCenters,NumBridgeheadAtoms,NumHAcceptors,NumHeteroatoms,NumHeterocycles,NumRotatableBonds,NumSaturatedCarbocycles,NumSaturatedHeterocycles,NumSaturatedRings,NumSpiroAtoms,NumUnspecifiedAtomStereoCenters,RingCount,MolLogP,fr_Al_COO,fr_Al_OH,fr_Al_OH_noTert,fr_ArN,fr_Ar_COO,fr_Ar_N,fr_Ar_NH,fr_Ar_OH,fr_COO,fr_C_S,fr_HOCCN,fr_Imine,fr_NH0,fr_NH1,fr_NH2,fr_N_O,fr_Ndealkylation1,fr_Ndealkylation2,fr_SH,fr_aldehyde,fr_alkyl_carbamate,fr_alkyl_halide,fr_allylic_oxid,fr_amidine,fr_aniline,fr_aryl_methyl,fr_azide,fr_azo,fr_bicyclic,fr_ester,fr_ether,fr_furan,fr_guanido,fr_hdrzine,fr_hdrzone,fr_imidazole,fr_imide,fr_isocyan,fr_ketone,fr_ketone_Topliss,fr_lactone,fr_methoxy,fr_morpholine,fr_nitro,fr_nitro_arom_nonortho,fr_oxazole,fr_oxime,fr_para_hydroxylation,fr_phos_acid,fr_piperdine,fr_piperzine,fr_priamide,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea,graph_diameter,avg_shortest_path,num_cycles
0,87817.0,*CC(*)c1ccccc1C(=O)OCCCCCC,NaN,0.374645,0.205667,0.932,NaN,12.144536,0.105927,-0.105927,0.500278,13.705882,232.323,1.411765,2.456411,2.563477,4.261290,5.852071,0.000000,0.000000,0.0,0.000000,0.000000,103.451541,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,9.531400,5.969305,0.0,0.000000,0.000000,45.951583,6.606882,35.392371,0.000000,0.000000,0.000000,0.000000,0.000000,12.576187,4.736863,0.000000,54.949285,6.923737,0.000000,26.30,0.000000,4.794537,0.0,5.969305,6.606882,24.825916,25.328832,0.000000,12.132734,19.056471,4.736863,6.120445,0.000000,12.144536,0.000000,2.105648,-0.105927,5.344954,4.148954,0.000000,0.533333,0,0,0,0,0,0,1,0,2,4,0,8,0,0,0,0,1,1,3.98170,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,12,4.736842,1
1,106919.0,*Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5...,NaN,0.370410,NaN,NaN,NaN,3.523412,0.098918,0.098918,0.125364,16.777778,598.919,0.577778,3.135512,1.451540,10.636376,14.715582,0.000000,0.000000,0.0,0.000000,0.000000,222.907896,0.000000,0.000000,0.0,0.0,32.607024,18.759549,0.000000,0.000000,0.000000,11.374773,0.0,0.000000,5.917906,115.071874,11.467335,130.442582,0.000000,11.467335,11.374773,0.000000,0.000000,0.000000,5.414990,5.917906,143.037592,0.000000,0.000000,24.06,0.000000,0.000000,0.0,5.414990,11.835812,5.917906,121.805341,0.000000,0.000000,129.300420,0.000000,0.326442,0.000000,0.000000,0.000000,18.579143,1.794202,15.456662,6.971589,0.000000,0.441860,2,1,0,1,0,0,2,0,2,4,0,16,1,0,1,0,0,5,12.35960,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,22,8.364477,5
2,388772.0,*Oc1ccc(S(=O)(=O)c2ccc(Oc3ccc(C4(c5ccc(Oc6ccc(...,NaN,0.378860,NaN,NaN,NaN,13.714745,0.107441,-3.829434,0.092387,16.301370,1003.207,0.397260,3.440031,0.743739,16.833145,19.899082,9.473726,22.998047,0.0,9.837253,0.000000,222.531582,0.000000,8.417797,0.0,0.0,43.527933,96.764951,5.414990,9.790967,40.947247,43.676159,0.0,0.000000,0.000000,76.363497,0.000000,227.523761,40.246583,25.383483,0.000000,40.246583,0.000000,22.618839,29.88403

# Data Preparation For Model Training 

In [17]:
# We'll separate train to be one model for each target variable.
t_1=train[['SMILES','Tg']].copy()
t_2=train[['SMILES','FFV']].copy()
t_3=train[['SMILES','Tc']].copy()
t_4=train[['SMILES','Density']].copy()
t_5=train[['SMILES','Rg']].copy()

# We will drop the rows with missing values related to that target after separation.
# This is important , dropping them beforehand would result Null for all data.
t_1.dropna(inplace=True)
t_2.dropna(inplace=True)
t_3.dropna(inplace=True)
t_4.dropna(inplace=True)
t_5.dropna(inplace=True)

# t_1.index = range(0, len(t_1))
# t_2.index = range(0, len(t_2))
# t_3.index = range(0, len(t_3))
# t_4.index = range(0, len(t_4))
# t_5.index = range(0, len(t_5))

In [18]:
# mols_test_t1 = [Chem.MolFromSmiles(s) for s in t_1.SMILES]
# mols_test_t2 = [Chem.MolFromSmiles(s) for s in t_2.SMILES]
# mols_test_t3 = [Chem.MolFromSmiles(s) for s in t_3.SMILES]
# mols_test_t4 = [Chem.MolFromSmiles(s) for s in t_4.SMILES]
# mols_test_t5 = [Chem.MolFromSmiles(s) for s in t_5.SMILES]
# # Initialize the Mordred Calculator
# calc = Calculator(descriptors, ignore_3D=True) # ignore_3D=True for 2D descriptors

# desc_t1 = calc.pandas(mols_test_t1)
# desc_t2 = calc.pandas(mols_test_t1)
# desc_t3 = calc.pandas(mols_test_t1)
# desc_t4 = calc.pandas(mols_test_t1)
# desc_t5 = calc.pandas(mols_test_t1)

In [19]:
# desc_t1['Tg']=t_1['Tg'].copy()
# desc_t2['FFV']=t_2['FFV'].copy()
# desc_t3['Tc']=t_3['Tc'].copy()
# desc_t4['Density']=t_4['Density'].copy()
# desc_t5['Rg']=t_5['Rg'].copy()

In [20]:
# desc_t1 = desc_t1.select_dtypes(exclude=['object', 'category'])
# desc_t2 = desc_t2.select_dtypes(exclude=['object', 'category'])
# desc_t3 = desc_t3.select_dtypes(exclude=['object', 'category'])
# desc_t4 = desc_t4.select_dtypes(exclude=['object', 'category'])
# desc_t5 = desc_t5.select_dtypes(exclude=['object', 'category'])

# tg_ds=pd.concat([tg_ds, desc_t1], ignore_index=True)
# ffv_ds=pd.concat([ffv_ds, desc_t2], ignore_index=True)
# tc_ds=pd.concat([tc_ds, desc_t3], ignore_index=True)
# den_ds=pd.concat([den_ds, desc_t4], ignore_index=True)
# rg_ds=pd.concat([rg_ds, desc_t5], ignore_index=True)
# # tg=t_1.merge(train,on='SMILES',how='left')
# # ffv=t_2.merge(train,on='SMILES',how='left')
# # tc=t_3.merge(train,on='SMILES',how='left')
# # density=t_4.merge(train,on='SMILES',how='left')
# # rg=t_5.merge(train,on='SMILES',how='left')

In [21]:
train=train.drop(['id','Tg','FFV','Tc','Density','Rg'],axis=1)
test=test.drop(['id','SMILES'],axis=1)

In [22]:
tg=t_1.merge(train,on='SMILES',how='left')
ffv=t_2.merge(train,on='SMILES',how='left')
tc=t_3.merge(train,on='SMILES',how='left')
density=t_4.merge(train,on='SMILES',how='left')
rg=t_5.merge(train,on='SMILES',how='left')

In [23]:
for i in (tg,tc,density,ffv,rg):
    i.drop('SMILES',axis=1,inplace=True)
    i.dropna(inplace=True)

In [24]:
tg.head()

,Tg,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,FpDensityMorgan1,AvgIpc,BalabanJ,Ipc,Kappa2,PEOE_VSA1,PEOE_VSA10,PEOE_VSA11,PEOE_VSA12,PEOE_VSA13,PEOE_VSA14,PEOE_VSA2,PEOE_VSA3,PEOE_VSA4,PEOE_VSA5,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,PEOE_VSA9,SMR_VSA1,SMR_VSA10,SMR_VSA2,SMR_VSA3,SMR_VSA4,SMR_VSA5,SMR_VSA6,SMR_VSA7,SMR_VSA9,SlogP_VSA1,SlogP_VSA10,SlogP_VSA11,SlogP_VSA12,SlogP_VSA2,SlogP_VSA3,SlogP_VSA4,SlogP_VSA5,SlogP_VSA7,SlogP_VSA8,TPSA,EState_VSA1,EState_VSA10,EState_VSA11,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,EState_VSA6,EState_VSA7,EState_VSA8,EState_VSA9,VSA_EState1,VSA_EState10,VSA_EState2,VSA_EState3,VSA_EState4,VSA_EState5,VSA_EState7,VSA_EState8,VSA_EState9,FractionCSP3,NHOHCount,NumAliphaticCarbocycles,NumAliphaticHeterocycles,NumAliphaticRings,NumAmideBonds,NumAromaticHeterocycles,NumAtomStereoCenters,NumBridgeheadAtoms,NumHAcceptors,NumHeteroatoms,NumHeterocycles,NumRotatableBonds,NumSaturatedCarbocycles,NumSaturatedHeterocycles,NumSaturatedRings,NumSpiroAtoms,NumUnspecifiedAtomStereoCenters,RingCount,MolLogP,fr_Al_COO,fr_Al_OH,fr_Al_OH_noTert,fr_ArN,fr_Ar_COO,fr_Ar_N,fr_Ar_NH,fr_Ar_OH,fr_COO,fr_C_S,fr_HOCCN,fr_Imine,fr_NH0,fr_NH1,fr_NH2,fr_N_O,fr_Ndealkylation1,fr_Ndealkylation2,fr_SH,fr_aldehyde,fr_alkyl_carbamate,fr_alkyl_halide,fr_allylic_oxid,fr_amidine,fr_aniline,fr_aryl_methyl,fr_azide,fr_azo,fr_bicyclic,fr_ester,fr_ether,fr_furan,fr_guanido,fr_hdrzine,fr_hdrzone,fr_imidazole,fr_imide,fr_isocyan,fr_ketone,fr_ketone_Topliss,fr_lactone,fr_methoxy,fr_morpholine,fr_nitro,fr_nitro_arom_nonortho,fr_oxazole,fr_oxime,fr_para_hydroxylation,fr_phos_acid,fr_piperdine,fr_piperzine,fr_priamide,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiophene,fr_unbrch_alkane,fr_urea,graph_diameter,avg_shortest_path,num_cycles
0,208.639749,11.248449,0.023312,-0.334079,0.435492,13.615385,185.183,1.384615,2.210448,3.709964,3.151338,3.993205,0.0,0.0,0.0,0.0,0.0,74.906467,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,14.383612,18.100520,0.0,16.367245,0.0,12.965578,13.089513,0.000000,0.000000,16.367245,0.0,0.000000,0.000000,37.231874,14.383612,0.0,6.923737,0.000000,0.000000,87.30,0.0,14.383612,0.0,37.231874,0.000000,0.000000,0.000000,6.923737,0.000000,16.367245,0.000000,-0.357391,0.000000,33.004449,4.941399,3.000789,-0.488941,0.133398,1.818204,0.000000,0.571429,3,0,0,0,2,0,1,0,4,8,0,6,0,0,0,0,1,0,-2.2629,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,4.285714,0
1,-41.266724,2.142025,0.580086,0.580086,0.318231,12.571429,268.538,0.714286,2.306121,2.828262,3.566187,10.902614,0.0,0.0,0.0,0.0,0.0,106.749724,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,44.042496,0.0,0.000000,0.0,45.448667,17.258561,0.000000,0.000000,0.000000,0.0,0.000000,44.042496,17.258561,0.000000,0.0,38.524930,6.923737,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,55.783491,6.923737,0.000000,44.042496,0.000000,1.277360,2.142025,0.000000,0.000000,0.000000,4.148997,8.837643,1.611538,6.315771,1.000000,0,0,0,0,0,0,0,0,4,6,0,13,0,0,0,0,0,0,5.5217,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,15,5.666667,0
2,-17.282022,2.408148,0.540428,0.540428,0.448030,13.600000,138.254,1.000000,2.098391,2.813864,2.689242,5.519191,0.0,0.0,0.0,0.0,0.0,64.524444,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,51.869489,0.000000,12.654956,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,44.945751,6.923737,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,38.524930,13.344559,0.000000,12.654956,0.000000,1.252280,0.000000,0.000000,0.000000,0.000000,0.000000,12.721612,4.026108,0.000000,0.800000,0,0,0,0,0,0,0,0,0,2,0,8,0,0,0,0,0,0,3.7477,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

# Model 

In [25]:

def model(train_d,test_d,model,target,submission=False):
    # We divide the data into training and validation sets for model evaluation
    train_cols = set(train_d.columns) - {target}
    test_cols = set(test_d.columns)
   # Intersect the feature columns
    common_cols = list(train_cols & test_cols)
    X=train_d[common_cols].copy()
    y=train_d[target].copy()
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=10)

    Model=model()
    if submission==False:
       Model.fit(X_train,y_train)
       y_pred=Model.predict(X_test)
       return mean_absolute_error(y_pred,y_test)         # We assess our model performance using MAE metric
    if submission==True:
       Model.fit(X,y)
       submission=Model.predict(test_d[common_cols].copy())
       return submission

# Data Selection

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
def select_features(dataset, target_column, n_features_to_select=500, estimator=None, step=12, random_state=42):
    
    # Make a copy to avoid modifying original data
    data = dataset.copy()
    
    # Separate features and target
    X = data.drop(target_column, axis=1)
    y = data[target_column]
    
    # Handle missing values - drop rows where target is NaN
    non_null_mask = y.notna()
    X = X[non_null_mask]
    y = y[non_null_mask]
    
    # Create estimator if not provided
    if estimator is None:
         estimator = CatBoostRegressor(
            iterations=100,
            learning_rate=0.03,
            depth=6,
            l2_leaf_reg=3,
            random_state=42,
            verbose=0
        )
    
    # Initialize RFE selector
    selector = RFE(
        estimator=estimator,
        n_features_to_select=n_features_to_select,
        step=step
    )
    
    # Fit the selector
    selector.fit(X, y)
    
    # Get selected features
    selected_features = X.columns[selector.support_].tolist()
    
    # Create new dataset with only selected features and target
    selected_data = data[selected_features + [target_column]]
    
    return selected_data

tg_ds=select_features(tg_ds, 'Tg')
ffv_ds=select_features(ffv_ds, 'FFV')
tc_ds=select_features(tc_ds, 'Tc')
den_ds=select_features(den_ds, 'Density')
rg_ds=select_features(rg_ds, 'Rg')

# **Model Evaluation

In [27]:

models_to_compare = {
    'XGBoost': lambda: XGBRegressor(verbosity=0),
    'CatBoost': lambda: CatBoostRegressor(verbose=0),
    'CatBoost2': lambda: CatBoostRegressor(iterations=1000,learning_rate=0.03,depth=6,l2_leaf_reg=3,random_seed=SEED,verbose=0),
    'MLP': lambda: MLPRegressor(
            hidden_layer_sizes=(1024, 512, 256),
            activation='relu',
            solver='adam',
            alpha=0.0001,
            batch_size=64,  
            learning_rate_init=0.0005, 
            max_iter=1000, 
            early_stopping=True, 
            random_state=42
        )
}

In [28]:
scaler = StandardScaler()
tg_ds = tg_ds.dropna(subset=['Tg'])
tg_ds_clean = tg_ds.dropna()
for name, model_init in models_to_compare.items():
    try:
        if name == 'MLP':
            score = model(tg_ds_clean, desc_test, model_init, 'Tg', submission=False)
        else:
            score = model(tg_ds, desc_test, model_init, 'Tg', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')
        
for name, model_init in models_to_compare.items():
    try:
        if name == 'MLP':
            score = model(tg, test, model_init, 'Tg', submission=False)
        else:
            score = model(tg, test, model_init, 'Tg', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')

XGBoost              : MAE = 27.07051
CatBoost             : MAE = 27.38890
CatBoost2            : MAE = 29.27032
MLP                  : MAE = 17.80769
XGBoost              : MAE = 39.50637
CatBoost             : MAE = 35.47715
CatBoost2            : MAE = 33.96786
MLP                  : MAE = 38.17630


In [29]:
ffv_ds = ffv_ds.dropna(subset=['FFV'])
ffv_ds_clean = ffv_ds.dropna()
for name, model_cls in models_to_compare.items():
    try:
        if name == 'MLP':
            score = model(ffv_ds_clean, desc_test, model_cls, 'FFV', submission=False)
        else:
            score = model(ffv_ds, desc_test, model_cls, 'FFV', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')
for name, model_init in models_to_compare.items():
    try:
        if name == 'MLP':
            score = model(ffv, test, model_init, 'FFV', submission=False)
        else:
            score = model(ffv, test, model_init, 'FFV', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')

XGBoost              : MAE = 0.00688
CatBoost             : MAE = 0.00614
CatBoost2            : MAE = 0.00663
MLP                  : Failed with error: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.
XGBoost              : MAE = 0.00727
CatBoost             : MAE = 0.00656
CatBoost2            : MAE = 0.00717
MLP                  : MAE = 0.07461


In [30]:
tc_ds = tc_ds.dropna(subset=['Tc'])
tc_ds_clean = tc_ds.dropna()
for name, model_cls in models_to_compare.items():
    try:
        if name == 'MLP':
            score = model(tc_ds_clean, desc_test, model_cls, 'Tc', submission=False)
        else:
            score = model(tc_ds, desc_test, model_cls, 'Tc', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')
for name, model_init in models_to_compare.items():
    try:
        if name == 'MLP':
            score = model(tc, test, model_init, 'Tc', submission=False)
        else:
            score = model(tc, test, model_init, 'Tc', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')

XGBoost              : MAE = 0.03696
CatBoost             : MAE = 0.03544
CatBoost2            : MAE = 0.03497
MLP                  : Failed with error: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.
XGBoost              : MAE = 0.03354
CatBoost             : MAE = 0.03400
CatBoost2            : MAE = 0.03541
MLP                  : MAE = 0.09929


In [31]:
den_ds = den_ds.dropna(subset=['Density'])
den_ds_clean = den_ds.dropna()
for name, model_cls in models_to_compare.items():
    try:
        if name == 'MLP':
            score = model(den_ds_clean, desc_test, model_cls, 'Density', submission=False)
        else:
            score = model(den_ds, desc_test, model_cls, 'Density', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')
for name, model_init in models_to_compare.items():
    try:
        if name == 'MLP':
            score = model(density, test, model_init, 'Density', submission=False)
        else:
            score = model(density, test, model_init, 'Density', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')

XGBoost              : MAE = 0.08967
CatBoost             : MAE = 0.07705
CatBoost2            : MAE = 0.07223
MLP                  : MAE = 150550.29490
XGBoost              : MAE = 0.04704
CatBoost             : MAE = 0.04046
CatBoost2            : MAE = 0.04112
MLP                  : MAE = 0.17305


In [32]:
rg_ds = rg_ds.dropna(subset=['Rg'])
rg_ds_clean = rg_ds.dropna()
for name, model_cls in models_to_compare.items():
    try:
        if name == 'MLP':
            score = model(rg_ds_clean, desc_test, model_cls, 'Rg', submission=False)
        else:
            score = model(rg_ds, desc_test, model_cls, 'Rg', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')
for name, model_init in models_to_compare.items():
    try:
        if name == 'MLP':
            score = model(rg, test, model_init, 'Rg', submission=False)
        else:
            score = model(rg, test, model_init, 'Rg', submission=False)
        print(f'{name:<21}: MAE = {score:.5f}')
    except Exception as e:
        print(f'{name:<21}: Failed with error: {e}')

XGBoost              : MAE = 1.64378
CatBoost             : MAE = 1.52381
CatBoost2            : MAE = 1.56659
MLP                  : Failed with error: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.
XGBoost              : MAE = 1.88121
CatBoost             : MAE = 1.79191
CatBoost2            : MAE = 1.80846
MLP                  : MAE = 2.27327


# Final Model For Submission

In [33]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
'''
11297.2s	273	=== Best models summary ===
11297.2s	274	Tg: CatBoost  CV_MAE=42.369263  params={'learning_rate': 0.03, 'l2_leaf_reg': 3, 'iterations': 1000, 'depth': 6}
11297.2s	275	FFV: XGBoost  CV_MAE=0.006229  params={'subsample': 0.8, 'reg_lambda': 1, 'reg_alpha': 0.0, 'n_estimators': 800, 'max_depth': 6, 'learning_rate': 0.05, 'colsample_bytree': 0.7}
11297.2s	276	Tc: LightGBM  CV_MAE=0.032098  params={'subsample': 0.6, 'reg_lambda': 1, 'reg_alpha': 0.001, 'num_leaves': 64, 'n_estimators': 500, 'learning_rate': 0.01, 'colsample_bytree': 0.5}
11297.2s	277	Density: GradientBoosting  CV_MAE=0.065598  params={'subsample': 0.8, 'n_estimators': 800, 'max_features': 'sqrt', 'max_depth': 3, 'learning_rate': 0.1}
11297.2s	278	Rg: GradientBoosting  CV_MAE=1.717341  params={'subsample': 0.8, 'n_estimators': 1000, 'max_features': 'sqrt', 'max_depth': 5, 'learning_rate': 0.01}
'''
desc_test_clean = desc_test.dropna(axis=1, how='any')
desc_test_clean.shape
models = {
    'XGBoost': lambda: XGBRegressor(verbosity=0),
    'CatBoost': lambda: CatBoostRegressor(verbose=0),
    'CatBoost2': lambda: CatBoostRegressor(iterations=1000,learning_rate=0.03,depth=6,l2_leaf_reg=3,random_seed=SEED,verbose=0),
    'MLP': lambda: MLPRegressor(
            hidden_layer_sizes=(1024, 512, 256),
            activation='relu',
            solver='adam',
            alpha=0.0001,
            batch_size=64,  
            learning_rate_init=0.0005, 
            max_iter=1000, 
            early_stopping=True, 
            random_state=42
        )
}

sub = {
    'id': ID,
    'Tg': model(tg_ds, desc_test_clean, models['CatBoost'], 'Tg', submission=True),
    'FFV': model(ffv_ds, desc_test, models['CatBoost'], 'FFV', submission=True),
    'Tc': model(tc, test, models['CatBoost2'], 'Tc', submission=True),
    'Density': model(density, test, models['CatBoost'], 'Density', submission=True),
    'Rg': model(rg_ds, desc_test, models['CatBoost'], 'Rg', submission=True)
}

In [34]:
submission=pd.DataFrame(sub)

In [35]:
desc_test_clean.shape

(3, 1584)

In [36]:
submission

,id,Tg,FFV,Tc,Density,Rg
0,1109053969,166.175612,0.369007,0.221261,1.151854,20.938248
1,1422188626,162.486858,0.375746,0.253123,1.091464,20.159403
2,2032016830,153.747002,0.355291,0.247329,1.093915,21.064611


In [37]:
submission.to_csv('submission.csv',index=False)